In [55]:
import pandas as pd
import numpy as np
import os

params = {
    "flow":"EXP",
    "year":"2019",
    "db-connector":"clickhouse-local"}

source_url = "http://www.mdic.gov.br/balanca/bd/comexstat-bd/mun/{}_{}_MUN.csv".format(params["flow"], params["year"])


In [56]:
if "{}_{}_MUN.csv".format(params["flow"], params["year"]) not in os.listdir("data_temp/"):
    df = pd.read_csv(source_url)
    df.to_csv("./data_temp/{}_{}_MUN.csv".format(params["flow"], params["year"]))
else:
    df = pd.read_csv("./data_temp/{}_{}_MUN.csv".format(params["flow"], params["year"]), sep=";")
    
df.head()

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB
0,2019,3,8541,493,SP,3410609,456,31240
1,2019,1,2710,169,PR,4125506,3230,8629
2,2019,2,7019,63,SP,3426902,3915,19960
3,2019,3,4010,756,SP,3409502,107,4129
4,2019,3,207,676,PR,4105508,362970,549491


In [57]:
df["time_id"] = (df["CO_ANO"].astype("str") + df["CO_MES"].astype("str").str.zfill(2)).astype("int")

df.head()

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,time_id
0,2019,3,8541,493,SP,3410609,456,31240,201903
1,2019,1,2710,169,PR,4125506,3230,8629,201901
2,2019,2,7019,63,SP,3426902,3915,19960,201902
3,2019,3,4010,756,SP,3409502,107,4129,201903
4,2019,3,207,676,PR,4105508,362970,549491,201903


In [58]:
p = pd.read_csv("./resources/shared_hs92.csv")
p_dict = {k:v for (k,v) in zip(p["hs4"],p["hs4_id"])}

df["hs4_id"] = df["SH4"].map(p_dict).fillna(33391).astype("int")

df.head()

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,time_id,hs4_id
0,2019,3,8541,493,SP,3410609,456,31240,201903,37469
1,2019,1,2710,169,PR,4125506,3230,8629,201901,52710
2,2019,2,7019,63,SP,3426902,3915,19960,201902,5947
3,2019,3,4010,756,SP,3409502,107,4129,201903,8474
4,2019,3,207,676,PR,4105508,362970,549491,201903,10207


In [59]:
g1 = pd.read_csv("./resources/PAIS.csv", sep=";", encoding="latin-1")
g1_dict = {k:v for (k,v) in zip(g1["CO_PAIS"],g1["CO_PAIS_ISOA3"])}

df["country_id"] = df["CO_PAIS"].map(g1_dict).str.lower()

df.head()

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,time_id,hs4_id,country_id
0,2019,3,8541,493,SP,3410609,456,31240,201903,37469,mex
1,2019,1,2710,169,PR,4125506,3230,8629,201901,52710,col
2,2019,2,7019,63,SP,3426902,3915,19960,201902,5947,arg
3,2019,3,4010,756,SP,3409502,107,4129,201903,8474,zaf
4,2019,3,207,676,PR,4105508,362970,549491,201903,10207,rus


In [60]:
g2 = pd.read_csv("./resources/shared_countries.csv")
g2_dict = {k:v for (k,v) in zip(g2["iso3"],g2["id_num"])}

df["country_id"] = df["country_id"].map(g2_dict)
df = df.dropna()
df["country_id"] = df["country_id"].astype("int")

df.head()

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,time_id,hs4_id,country_id
0,2019,3,8541,493,SP,3410609,456,31240,201903,37469,484.0
1,2019,1,2710,169,PR,4125506,3230,8629,201901,52710,170.0
2,2019,2,7019,63,SP,3426902,3915,19960,201902,5947,32.0
3,2019,3,4010,756,SP,3409502,107,4129,201903,8474,711.0
4,2019,3,207,676,PR,4105508,362970,549491,201903,10207,643.0


In [61]:
s = pd.read_csv("./resources/UF.csv", sep=";", encoding="latin-1")
s_dict = {k:v for (k,v) in zip(s["SG_UF"],s["CO_UF"])}

df["state_id"] = df["SG_UF_MUN"].map(s_dict)

df.head()

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,time_id,hs4_id,country_id,state_id
0,2019,3,8541,493,SP,3410609,456,31240,201903,37469,484.0,41
1,2019,1,2710,169,PR,4125506,3230,8629,201901,52710,170.0,42
2,2019,2,7019,63,SP,3426902,3915,19960,201902,5947,32.0,41
3,2019,3,4010,756,SP,3409502,107,4129,201903,8474,711.0,41
4,2019,3,207,676,PR,4105508,362970,549491,201903,10207,643.0,42


In [62]:
df["municipality_id"] = df["CO_MUN"]
df["liquid_kg"] = df["KG_LIQUIDO"]
df["fob"] = df["VL_FOB"]

df.head()

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,time_id,hs4_id,country_id,state_id,municipality_id,liquid_kg,fob
0,2019,3,8541,493,SP,3410609,456,31240,201903,37469,484.0,41,3410609,456,31240
1,2019,1,2710,169,PR,4125506,3230,8629,201901,52710,170.0,42,4125506,3230,8629
2,2019,2,7019,63,SP,3426902,3915,19960,201902,5947,32.0,41,3426902,3915,19960
3,2019,3,4010,756,SP,3409502,107,4129,201903,8474,711.0,41,3409502,107,4129
4,2019,3,207,676,PR,4105508,362970,549491,201903,10207,643.0,42,4105508,362970,549491


In [63]:
f_dict = {"EXP":1,
          "IMP":2}
df["flow_id"] = f_dict[params["flow"]]

df = df[["time_id","flow_id","hs4_id","country_id","state_id","municipality_id","liquid_kg","fob"]]

len(df)

316237